# 运算符重载

Python支持运算符重载。并且通过对运算符重载施加一些限制，Python的运算符重载特性在灵活性、可用性以及安全性之间取得了一定的平衡。

具体来说，Python对运算符重载有如下限制：
1. 不能重载内置类型的运算符
2. 不能新建运算符
3. 某些运算符不能重载 —— is、and、or以及not（位运算符&、|以及~可以重载）

从实际使用中来看，上述规则实际上和Python特别强调duck type是相互协调的 —— Python中运算符重载功能的实现基于自定义类中对应方法的实现，即若想重载某一个运算符，则需要实现对应的方法。

由于没有办法覆写内置类型某些方法的实现，因此无法重载内置类型的运算符。

不能重载的运算符则不是通过上述机制运作的，因此也没有办法重载。下述例子查看了进行and、or以及not运算的字节码。显然，这些运算符并没有调用任何方法。and被翻译为字节码JUMP_IF_FALSE_OR_POP，or被翻译为JUMP_IF_TRUE_OR_POP，not则被翻译为UNARY_NOT。

In [29]:
# 尝试覆盖list类型的__len__方法
try:
    list.__len__ = 42
except TypeError:
    print("TypeError")

# 查看and语句的字节码
import dis
dis.opmap
def and_op_test(value_1, value_2):
    return not ((value_1<0 and value_2<0) or (value_1 != value_2))

dis.dis(and_op_test)

TypeError
 11           0 LOAD_FAST                0 (value_1)
              2 LOAD_CONST               1 (0)
              4 COMPARE_OP               0 (<)
              6 POP_JUMP_IF_FALSE       16
              8 LOAD_FAST                1 (value_2)
             10 LOAD_CONST               1 (0)
             12 COMPARE_OP               0 (<)
             14 JUMP_IF_TRUE_OR_POP     22
        >>   16 LOAD_FAST                0 (value_1)
             18 LOAD_FAST                1 (value_2)
             20 COMPARE_OP               3 (!=)
        >>   22 UNARY_NOT
             24 RETURN_VALUE


## 一元运算符的重载

## 增量赋值运算符的特殊性

## 二元运算符的重载

## 比较运算符的特殊性

## 总结